In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy import stats
import calibration as cal

In [2]:
c1 = ['clock', 'knife', 'oven', 'chair', 'bottle', 'keyboard']
c2 = ['cat', 'elephant', 'dog', 'bird', 'bear']
c3 = ['airplane', 'boat', 'car', 'truck', 'bicycle']

def convert_to_tri_class(x, c1, c2, c3):
    if x in c1:
        c = 0
    elif x in c2:
        c = 1
    else:
        assert x in c3
        c = 2
    return c

def convert_prob_to_tri_class(row, c1, c2, c3):
    c1_sum = 0
    for v in c1:
        c1_sum += row[v]
    c2_sum = 0
    for v in c2:
        c2_sum += row[v]
    c3_sum = 0
    for v in c3:
        c3_sum += row[v]
    row['model_p0'] = min(1.0, c1_sum)
    row['model_p1'] = min(1.0, c2_sum)
    row['model_p2'] = min(1.0, c3_sum)
    return row

def combine_experts(rows):
    true_result = rows.iloc[0]['image_category_new']
    expert_predictions = rows['participant_classification_new']
    
    correct_responses = []
    incorrect_responses = []
    for pred in expert_predictions:
        if pred==true_result:
            correct_responses.append(pred)
        else:
            incorrect_responses.append(pred)
            
    expert_predictions = []
    if len(correct_responses)>=2:
        expert_predictions = correct_responses[:2]
        if len(incorrect_responses)>=1:
            expert_predictions.append(incorrect_responses[0])
        else:
            expert_predictions.append(correct_responses[2])
    elif len(correct_responses)==1:
        expert_predictions = [correct_responses[0]]
        expert_predictions.extend(incorrect_responses[:2])
    else:
        expert_predictions = incorrect_responses[:3]
    if true_result == 1:
        expert_predictions = [expert_predictions[2], expert_predictions[0], expert_predictions[1]]
    elif true_result == 2:
        expert_predictions = [expert_predictions[1], expert_predictions[2], expert_predictions[0]]
        
    return expert_predictions

In [10]:
# create dataframe of expert and model predictions
n_experts = 3
noisy = False
distribution_shift = False
df_model = pd.read_csv('model_preds_raw.csv')
df_human = pd.read_csv("annotations_raw.csv")

if not distribution_shift:
    # take subset of noisier images
    df_human = df_human[df_human['noise_level']>95]
    df_model = df_model[df_model['noise_level']>95]

df_human = df_human[[
 'participant_id', 'image_id', 'image_name', 'noise_level', 'image_category',
 'participant_classification', 'confidence', 'correct', 'total_accuracy'
]]
for c in ['participant_classification', 'image_category']:
    df_human[c+"_new"] = df_human[c].apply(
        convert_to_tri_class, args=(c1, c2, c3,)
    )

df_human_proc = pd.DataFrame(
    df_human.groupby(['image_name', 'noise_level']).apply(combine_experts),
    columns=["Y_H"]
)
df_human_proc.reset_index(inplace=True)
df_human_proc['consensus'] = df_human_proc['Y_H'].apply(
    lambda x: stats.mode(x)[0][0] if stats.mode(x)[1][0] > 1 else -1
)
for e in range(n_experts):
    df_human_proc['expert'+str(e+1)] = df_human_proc['Y_H'].apply(lambda x: x[e])
    
df_model = df_model.apply(
        convert_prob_to_tri_class, args=(c1, c2, c3,), axis=1
)
model_name = 'alexnet' if noisy else 'vgg19'
dn_df = df_model[df_model['model_name']==model_name].copy()
dn_df['model_pred_int'] = dn_df.apply(
    lambda x: np.argmax([x['model_p'+str(i)] for i in range(3)]), axis=1
)
dn_df = dn_df[['image_name', 'noise_level','model_p0','model_p1', 'model_p2','model_pred_int']]

df = df_human_proc.merge(dn_df, on=['image_name', 'noise_level'], how='inner')
df = df[df['consensus']!=-1].reset_index(drop=True)

# shuffle rows
np.random.seed(1)
order = np.array([i for i in range(len(df))])
np.random.shuffle(order)
df = df.loc[order]
df.reset_index(inplace=True, drop=True)

if distribution_shift:
    df_before_ds = df[df['noise_level']==80][:375]
    df_after_ds = df[df['noise_level']==125][:375]
    df = pd.concat([
        df_before_ds[:125], 
        df_after_ds[:125], 
        df_before_ds[125:250], 
        df_after_ds[125:250],
        df_before_ds[250:],
        df_after_ds[250:]
    ])
    
if noisy==False and distribution_shift==False:
    df.to_csv('imagenet_processed.csv')

/var/folders/2t/tpzk6y551s5bvtpwm5msn6740000gn/T/ipykernel_1016/2789766101.py:28: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  lambda x: stats.mode(x)[0][0] if stats.mode(x)[1][0] > 1 else -1


In [11]:
n = 750

In [12]:
df['model_correct'] = df['model_pred_int']==df['consensus']
test_accuracy = np.mean(df[:n]['model_correct'])
class_wise_accs = list(df[:n].groupby('consensus').aggregate(
        {'model_correct':'mean'}
)['model_correct'])
print("\tclassifier (overall): {}".format(test_accuracy))
print("\t\t " + str(class_wise_accs))

	classifier (overall): 0.9306666666666666
		 [0.9124087591240876, 0.95, 0.9322033898305084]


In [13]:
# get accuracies
n = 250
print('accuracy on test set (n={}):'.format(n))

df['model_correct'] = df['model_pred_int']==df['consensus']
test_accuracy = np.mean(df[:n]['model_correct'])
class_wise_accs = list(df[:n].groupby('consensus').aggregate(
        {'model_correct':'mean'}
)['model_correct'])
print("\tclassifier (overall): {}".format(test_accuracy))
print("\t\t " + str(class_wise_accs))
probs = np.array(df[['model_p0','model_p1','model_p2']])
ece = cal.get_ece(probs, df['consensus'])
cal_error = cal.get_calibration_error(probs, df['consensus'])
print("\t\tECE = {}; cal error = {}".format(ece, cal_error))

for e in range(n_experts):
    e_corr_col = 'expert{}_correct'.format(e+1)
    df[e_corr_col] = df['expert'+str(e+1)]==df['consensus']
    expert_acc = sum(df[:n]['expert'+str(e+1)]==df[:n]['consensus'])/n
    class_wise_accs = list(df[:n].groupby('consensus').aggregate(
            {e_corr_col:'mean'}
    )[e_corr_col])
    print ("\texpert {}: {}".format(e+1, expert_acc))
    print("\t\t " + str(class_wise_accs))

accuracy on test set (n=250):
	classifier (overall): 0.916
		 [0.9157894736842105, 0.922077922077922, 0.9102564102564102]
		ECE = 0.008347642802963608; cal error = 0.045566160989114375
	expert 1: 0.82
		 [1.0, 0.44155844155844154, 0.9743589743589743]
	expert 2: 0.888
		 [1.0, 1.0, 0.6410256410256411]
	expert 3: 0.828
		 [0.5684210526315789, 0.974025974025974, 1.0]


In [14]:
# save data dict for our model
y_h = np.array(df[['expert1','expert2','expert3']]) + 1
y_m = [[df['model_p'+str(i)]] for i in range(3)]
y_m = np.array(y_m).reshape((len(df), 1, 3))

out_dict = {
    'Y_H' : y_h.tolist(),
    'Y_M' : y_m.tolist(),
    'n_models': 1,
    'n_humans': n_experts,
    'K': 3
}

ext = ""
if noisy:
    ext += "_noisy"
if distribution_shift:
    ext += "_ds" 
    
with open('data' + ext + '.pickle', 'wb') as handle:
    pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
# create data dict for INFEXP model
for start_point in [0, 250, 500]:
    y_h = np.array(df[['expert'+str(i+1) for i in range(n_experts)]])
    y_h = y_h.reshape((n_experts,len(df)))
    d_new = np.array(df['consensus'])

    y_m_new = np.array([df['model_p'+str(i)] for i in range(3)])
    model_confs = np.array([y_m_new])
    model_preds = np.array([[np.argmax(i) for i in j] for j in model_confs])

    df['model_correct'] = df['model_pred_int']==df['consensus']
    model_perf = np.array([[df['model_correct'].mean()]])
    class_wise_perf = np.array(
        df.groupby(
            'consensus'
        ).aggregate(
            {'model_correct':'mean'}
        )['model_correct']
    )

    n_models = 1
    n_tests = 250
    infexp_dict = {
        'model_confs' : model_confs[:,start_point:start_point+n_tests],
        'model_preds' : model_preds[:,start_point:start_point+n_tests],
        'true_targets' : d_new[start_point:start_point+n_tests],
        'expert_preds' : y_h[:,start_point:start_point+n_tests],
        'chosen_models' : np.array([0]),
        'model_perf' : model_perf,
        'model_perf_per_class' : class_wise_perf
    }

    with open('imagenet_infexp{}.pickle'.format(start_point), 'wb') as handle:
        pickle.dump(infexp_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)